In [4]:
import os
from os import listdir
from os.path import isfile, join

import numpy as np
from nltk.util import ngrams
from sklearn.model_selection import train_test_split

import tensorflow as tf
from keras import backend as K
from keras.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.layers import Dropout, Dense, Flatten, BatchNormalization, LSTM, Input, concatenate
from keras.models import Sequential, Model, load_model
from keras.optimizers import Nadam

# Data Prep

In [5]:
def get_file_names(mypath):
    return [f for f in listdir(mypath) if isfile(join(mypath, f))]


def get_notes(notes_path):
    all_song_notes = []
    file_names = get_file_names(notes_path)
    
    for file_name in file_names:
        file_notes = []
        with open(notes_path + file_name, 'r') as file:
            lines = file.readlines()
            file_notes = np.asarray([x.replace("\n", "").split(" ") for x in lines[3:]]).astype("float32")
        all_song_notes.append(file_notes)
    return np.asarray(all_song_notes)


notes_by_song = get_notes("timings/")

In [118]:
def get_binary_rep(arrow_values):
    return (((np.asarray(arrow_values).astype(int)[:,None] & (1 << np.arange(4)))) > 0).astype(int)


def create_tokens(timings):
    tokens = np.zeros((timings.shape[0], 3))
    tokens[0][0] = 1 # set start token
    next_note_token = np.append(timings[1:] - timings[:-1], np.asarray([0]))
    prev_note_token = np.append(np.asarray([0]),  next_note_token[: -1])
    tokens[:, 1] = prev_note_token.reshape(1, -1)
    tokens[:, 2] = next_note_token.reshape(1, -1)
    return tokens.astype("float32")


def get_notes_ngram(binary_notes, lookback):
    padding = np.zeros((lookback, binary_notes.shape[1]))
    data_w_padding = np.append(padding, binary_notes, axis = 0)
    return np.asarray(list(ngrams(data_w_padding, lookback)))


def data_prep(notes_by_song, lookback = 5):
    from sklearn.preprocessing import OneHotEncoder
    encoder = OneHotEncoder(categories='auto', sparse = False).fit(np.asarray(range(1, 16)).reshape(-1, 1))
    
    all_arrows = []
    all_tokens = []
    all_labels = []
    
    for notes in notes_by_song:
        binary_notes = get_binary_rep(notes[:, 0][:-1])
        notes_ngram = get_notes_ngram(binary_notes, lookback)
        tokens = create_tokens(notes[:, 1])
        labels = encoder.transform(notes[:, 0].reshape(-1, 1))
        
        all_arrows.append(notes_ngram)
        all_tokens.append(tokens)
        all_labels.append(labels)
        
    return np.concatenate(all_arrows), np.concatenate(all_tokens), np.concatenate(all_labels)

lookback = 64
all_arrows, all_tokens, all_labels = data_prep(notes_by_song, lookback = lookback)
all_arrows.shape, all_tokens.shape, all_labels.shape

((24440, 64, 4), (24440, 3), (24440, 15))

# Training

In [147]:
arrows_train, arrows_test, labels_train, labels_test = \
    train_test_split(all_arrows, all_labels, test_size=0.2, random_state = 42, shuffle = True)

tokens_train, tokens_test, _, _ = \
    train_test_split(all_tokens, all_labels, random_state = 42, test_size=0.2, shuffle = True)

#    train_test_split(np.expand_dims(all_tokens, axis = 1), all_labels, random_state = 42, test_size=0.2, shuffle = False)
#   train_test_split(all_tokens, all_labels, random_state = 42, test_size=0.2, shuffle = True)
#    train_test_split(np.expand_dims(all_tokens, axis = 1), all_labels, random_state = 42, test_size=0.2, shuffle = True)

(arrows_train.shape, tokens_train.shape, labels_train.shape), (arrows_test.shape, tokens_test.shape, labels_test.shape)


(((19552, 64, 4), (19552, 3), (19552, 15)),
 ((4888, 64, 4), (4888, 3), (4888, 15)))

In [149]:
def build(arrows_shape, token_shape, output_shape, silent = True):
    arrows = Input(shape = (arrows_shape[1], arrows_shape[2],))
    tokens = Input(shape = (token_shape[1],))
    #tokens = Input(shape = (token_shape[1], token_shape[2],))
    
    x = LSTM(128, kernel_initializer='glorot_normal', return_sequences = True)(arrows)
    x = LSTM(128, kernel_initializer='glorot_normal')(x)
    x = Model(inputs = arrows, outputs = x)
    
    y = Dense(64, kernel_initializer='glorot_normal', activation = "relu")(tokens)
    #y = LSTM(128, kernel_initializer='glorot_normal')(y)
    y = Model(inputs = tokens, outputs = y)
    
    combined = concatenate([x.output, y.output])
    
    z = Dense(128, kernel_initializer='glorot_normal', activation = "relu")(combined)
    z = Dropout(0.5)(z)
    z = Dense(output_shape[1], activation = "softmax")(z)
    
    model = Model(inputs = [x.input, y.input], outputs = z)
    model.compile(loss='categorical_crossentropy', optimizer='Nadam', metrics=['accuracy'])
    
    if not silent: 
        model.summary()
    
    return model

In [150]:
model = build(arrows_train.shape, tokens_train.shape, labels_train.shape, silent = False)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_42 (InputLayer)           (None, 64, 4)        0                                            
__________________________________________________________________________________________________
lstm_71 (LSTM)                  (None, 64, 128)      68096       input_42[0][0]                   
__________________________________________________________________________________________________
input_43 (InputLayer)           (None, 3)            0                                            
__________________________________________________________________________________________________
lstm_72 (LSTM)                  (None, 128)          131584      lstm_71[0][0]                    
__________________________________________________________________________________________________
dense_39 (

In [151]:
batch_size = 256
model = build(arrows_train.shape, tokens_train.shape, labels_train.shape, silent = True)

callbacks = [EarlyStopping(monitor='val_loss', patience=5, verbose=0),
             ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.001)]

history = model.fit([arrows_train, tokens_train], 
                    labels_train,
                    validation_data=([arrows_test, tokens_test], labels_test),
                    epochs = 100,
                    callbacks = callbacks,
                    batch_size= batch_size,
                    verbose = 1)

Train on 19552 samples, validate on 4888 samples
Epoch 1/100
19552/19552 [==============================] - 33s 2ms/step - loss: 1.9011 - acc: 0.2283 - val_loss: 1.7279 - val_acc: 0.2238
Epoch 2/100
 5120/19552 [======>.......................] - ETA: 17s - loss: 1.7480 - acc: 0.2537

KeyboardInterrupt: 

# Predicting

In [53]:
model.save("arrow_placement_model_64.h5")

In [131]:
notes_ngram.shape

(1, 64, 4)

In [124]:
pred_notes = []
ex_timings = notes_by_song[0][:, 1]
ex_tokens = np.expand_dims(np.expand_dims(create_tokens(ex_timings), axis = 1), axis = 1)
notes_ngram = get_notes_ngram(np.zeros((1, 4)), lookback)[-1]
for i, token in enumerate(ex_tokens):
    pred_arrow = np.argmax(model.predict([notes_ngram, token], batch_size = 256)) + 1
    pred_notes.append(pred_arrow) 
    binary_note = get_binary_rep([pred_arrow])
    notes_ngram = np.roll(notes_ngram, -1, axis = 0)
    notes_ngram[0][-1] = binary_note
    print(notes_ngram[0][-1])

[1. 0. 0. 1.]
[1. 0. 0. 1.]
[1. 0. 0. 1.]
[1. 0. 0. 1.]
[0. 0. 1. 0.]
[0. 1. 0. 0.]
[0. 0. 1. 0.]
[0. 0. 1. 0.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0. 0. 1.]
[1. 0. 0. 0.]
[0. 0.

In [108]:
pred_notes

[9,
 9,
 9,
 9,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 1,
 8,
 9,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
 2,
 4,
